In [1]:
import pandas as pd
# Set option to display all columns
pd.set_option('display.max_columns', None)


In [2]:
data = pd.read_pickle("outputs_new/dedupe_data_group_merge.pkl")
data

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
0,crossref::10.21467/preprints.48,AIJR Preprints,crossref,10.21467/preprints.48,https://doi.org/10.21467/preprints.48,https://preprints.aijr.org/index.php/ap/prepri...,"Bird’s Eye View on the Diagnosis, Treatment, &...","Panchalingala, Sai Bhargavi",None,None,None,None,,,,,false,None,None,None,None,parent,2020-05-03,2020,parent,None,None,None,None,None,None,parent,None,None,parent,None,None
1,crossref::10.21467/preprints.43,AIJR Preprints,crossref,10.21467/preprints.43,https://doi.org/10.21467/preprints.43,https://preprints.aijr.org/index.php/ap/prepri...,Doxycycline and Minocycline Drugs as a Treatme...,"Mostafa, Mohamed",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-25,2020,parent,None,None,mohamed_mostafa,doxycycline and minocycline drugs as a treatme...,mostafa|m,None,parent,None,None,parent,None,None
2,crossref::10.21467/preprints.39,AIJR Preprints,crossref,10.21467/preprints.39,https://doi.org/10.21467/preprints.39,https://preprints.aijr.org/index.php/ap/prepri...,A Genetic Perspective of 2019-nCoV in Relation...,"Dasgupta, Rimjhim",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-16,2020,parent,None,None,dasgupta_rimjhim,a genetic perspective of 2019 ncov in relation...,dasgupta|r,None,parent,None,None,parent,None,None
3,crossref::10.21467/preprints.38,AIJR Preprints,crossref,10.21467/preprints.38,https://doi.org/10.21467/preprints.38,https://preprints.aijr.org/index.php/ap/prepri...,Marine Algae as a Natural Source for Antiviral...,"Musale, Amar S; G., Raja Krishna Kumar; Sapre,...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,parent,None,None,None,None,None,None,parent,None,None,parent,None,None
4,crossref::10.21467/preprints.36,AIJR Preprints,crossref,10.21467/preprints.36,https://doi.org/10.21467/preprints.36,https://preprints.aijr.org/index.php/ap/prepri...,Possible Prevention of COVID 19 by Using Linol...,"Subhash, Venkata; G, Raja Krishna Kumar; Sapre...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,parent,None,None,None,None,None,None,parent,None,None,parent,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084855,openalex::W999325625,viXra,openalex,None,None,https://vixra.org/pdf/1409.0090v1.pdf,Three Objections to Modern Physics,Lubomir Vlcek,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-09-01,2014,parent,None,None,lubomir_vlcek,three objections to modern physics,vlcek|l,None,parent,None,None,parent,None,None
8084856,openalex::W999460032,viXra,openalex,None,None,https://vixra.org/abs/1112.0094,Particle Mass Ratios,DT Froedge,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2011-12-01,2011,parent,None,None,dt_froedge,particle mass ratios,froedge|d,None,parent,None,None,parent,None,None
8084857,openalex::W99967155,viXra,openalex,None,None,https://vixra.org/pdf/1406.0019v1.pdf,Quantum FFF Theory Proposals for Some Unsolved...,Leo Vuyk,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-06-01,2014,parent,None,None,leo_vuyk,quantum fff theory proposals for some unsolved...,vuyk|l,None,parent,None,None,parent,None,None
8084858,openalex::W999790414,viXra,open

In [11]:
import os
import duckdb
import pandas as pd

def build_parent_index_and_long_full_duckdb(
    df: pd.DataFrame,
    *,
    out_dir: str = "outputs",
    record_id_col: str = "record_id",
    server_col: str = "server_name",
    doi_col: str = "doi",
    date_col: str = "date_first_seen",
    year_col: str = "publication_year_first_seen",
    group_id_col: str = "dup_group_id",
    # outputs
    long_full_name: str = "dedupe_clusters_long_full.parquet",
    parent_index_name: str = "parent_version_index_full.parquet",
    metrics_name: str = "dedupe_full_metrics.json",
):
    """
    Rules:
      - singleton rows: parent = itself
      - clustered rows: parent = oldest in cluster (date_col ASC, tie record_id ASC)
      - group id for full universe: dup_group_id_full = COALESCE(dup_group_id, record_id)

    Outputs (in out_dir):
      - dedupe_clusters_long_full.parquet  (ALL rows, includes parent_record_id_selected, is_parent_selected, group_size)
      - parent_version_index_full.parquet  (ONE row per group, with parent info + members lists)
      - dedupe_full_metrics.json
    """
    os.makedirs(out_dir, exist_ok=True)
    con = duckdb.connect(database=":memory:")
    con.execute("PRAGMA threads=8;")          # tune if needed
    con.execute("PRAGMA memory_limit='16GB';")# tune if needed

    # Register dataframe
    con.register("df_in", df)

    # -------------
    # 0) Build a stable "full" group id (clusters + singletons)
    # -------------
    # Note: we compute parent per group using window functions.
    # Parent selection: MIN_BY(record_id, sort_key) doesn't exist in DuckDB,
    # but we can do FIRST_VALUE over an ordered window.
    con.execute(f"""
        CREATE OR REPLACE TEMP VIEW base AS
        SELECT
            *,
            COALESCE(CAST({group_id_col} AS VARCHAR), CAST({record_id_col} AS VARCHAR)) AS dup_group_id_full,
            TRY_CAST({date_col} AS TIMESTAMP) AS dt
        FROM df_in
    """)

    # -------------
    # 1) Create long_full (ALL rows)
    #    parent_record_id_selected = oldest record_id within dup_group_id_full
    # -------------
    con.execute(f"""
        CREATE OR REPLACE TEMP VIEW long_full AS
        SELECT
            -- keep original columns
            b.*,

            -- selected parent per group
            FIRST_VALUE(CAST({record_id_col} AS VARCHAR)) OVER (
                PARTITION BY dup_group_id_full
                ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
            ) AS parent_record_id_selected,

            -- parent metadata (from the chosen parent row)
            FIRST_VALUE(CAST({server_col} AS VARCHAR)) OVER (
                PARTITION BY dup_group_id_full
                ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
            ) AS parent_server_name_selected,

            FIRST_VALUE(CAST({doi_col} AS VARCHAR)) OVER (
                PARTITION BY dup_group_id_full
                ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
            ) AS parent_doi_selected,

            FIRST_VALUE(dt) OVER (
                PARTITION BY dup_group_id_full
                ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
            ) AS parent_date_selected,

            FIRST_VALUE(CAST({year_col} AS VARCHAR)) OVER (
                PARTITION BY dup_group_id_full
                ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
            ) AS parent_year_selected,

            -- is this row the selected parent?
            CASE
              WHEN CAST({record_id_col} AS VARCHAR) =
                   FIRST_VALUE(CAST({record_id_col} AS VARCHAR)) OVER (
                      PARTITION BY dup_group_id_full
                      ORDER BY dt ASC NULLS LAST, CAST({record_id_col} AS VARCHAR) ASC
                   )
              THEN TRUE ELSE FALSE
            END AS is_parent_selected,

            COUNT(*) OVER (PARTITION BY dup_group_id_full) AS group_size
        FROM base b
    """)

    # Export full long table
    long_path = os.path.join(out_dir, long_full_name)
    con.execute(f"COPY long_full TO '{long_path}' (FORMAT 'parquet');")

    # -------------
    # 2) Build parent_version_index_full (ONE row per group)
    #    Includes lists (record_ids, dois, servers with counts), first/last version dates, etc.
    # -------------
    # a) servers with counts as "ServerA (12); ServerB (3)"
    con.execute(f"""
        CREATE OR REPLACE TEMP VIEW servers_counts AS
        SELECT
            dup_group_id_full,
            CAST({server_col} AS VARCHAR) AS server_name,
            COUNT(*) AS n
        FROM long_full
        GROUP BY 1, 2
    """)

    con.execute("""
        CREATE OR REPLACE TEMP VIEW servers_counts_str AS
        SELECT
            dup_group_id_full,
            STRING_AGG(server_name || ' (' || CAST(n AS VARCHAR) || ')', '; ' ORDER BY n DESC, server_name ASC) AS servers_with_counts
        FROM servers_counts
        GROUP BY 1
    """)

    # b) most recent version server(s): if multiple on same max date → join with ';'
    con.execute("""
        CREATE OR REPLACE TEMP VIEW max_dt AS
        SELECT dup_group_id_full, MAX(dt) AS max_dt
        FROM long_full
        GROUP BY 1
    """)

    con.execute("""
        CREATE OR REPLACE TEMP VIEW most_recent_servers AS
        SELECT
            l.dup_group_id_full,
            STRING_AGG(DISTINCT CAST(l.server_name AS VARCHAR), '; ' ORDER BY CAST(l.server_name AS VARCHAR)) AS most_recent_server_names
        FROM long_full l
        JOIN max_dt m
          ON l.dup_group_id_full = m.dup_group_id_full
         AND l.dt = m.max_dt
        GROUP BY 1
    """)

    # c) parent index
    parent_path = os.path.join(out_dir, parent_index_name)
    con.execute(f"""
        COPY (
            SELECT
                dup_group_id_full,

                -- parent (selected oldest)
                parent_record_id_selected AS parent_record_id,
                parent_doi_selected       AS parent_doi,
                parent_server_name_selected AS parent_server_name,
                parent_date_selected      AS parent_date_first_seen,
                parent_year_selected      AS parent_year_first_seen,

                -- cluster members lists (ALL of them, not just 30)
                STRING_AGG(DISTINCT CAST({record_id_col} AS VARCHAR), '; ' ORDER BY CAST({record_id_col} AS VARCHAR)) AS version_record_ids,
                STRING_AGG(DISTINCT CAST({doi_col} AS VARCHAR), '; ' ORDER BY CAST({doi_col} AS VARCHAR)) FILTER (WHERE {doi_col} IS NOT NULL AND CAST({doi_col} AS VARCHAR) <> '') AS version_dois,

                -- server list with counts
                s.servers_with_counts,

                -- size + first/last version date
                MAX(group_size) AS total_versions,
                MIN(dt) AS first_version_date,
                MAX(dt) AS last_version_date,

                -- most recent version server(s)
                mr.most_recent_server_names AS most_recent_version_servers

            FROM long_full l
            LEFT JOIN servers_counts_str s USING(dup_group_id_full)
            LEFT JOIN most_recent_servers mr USING(dup_group_id_full)
            GROUP BY
                dup_group_id_full,
                parent_record_id_selected,
                parent_doi_selected,
                parent_server_name_selected,
                parent_date_selected,
                parent_year_selected,
                s.servers_with_counts,
                mr.most_recent_server_names
        )
        TO '{parent_path}' (FORMAT 'parquet');
    """)

    # -------------
    # 3) Metrics
    # -------------
    total_rows = con.execute("SELECT COUNT(*) FROM base").fetchone()[0]
    clustered_rows = con.execute(f"SELECT COUNT(*) FROM base WHERE {group_id_col} IS NOT NULL").fetchone()[0]
    singleton_rows = total_rows - clustered_rows

    clusters = con.execute(f"SELECT COUNT(DISTINCT CAST({group_id_col} AS VARCHAR)) FROM base WHERE {group_id_col} IS NOT NULL").fetchone()[0]
    groups_full = con.execute("SELECT COUNT(DISTINCT dup_group_id_full) FROM base").fetchone()[0]

    # sanity checks
    long_rows = con.execute("SELECT COUNT(*) FROM long_full").fetchone()[0]
    parent_rows = con.execute("SELECT COUNT(DISTINCT dup_group_id_full) FROM long_full").fetchone()[0]

    metrics = {
        "total_rows": int(total_rows),
        "clustered_rows_has_dup_group_id": int(clustered_rows),
        "singleton_rows_no_dup_group_id": int(singleton_rows),
        "clusters_distinct_dup_group_id": int(clusters),
        "groups_full_clusters_plus_singletons": int(groups_full),
        "long_full_rows_written": int(long_rows),
        "parent_index_rows_written": int(parent_rows),
    }

    # write metrics json
    import json
    metrics_path = os.path.join(out_dir, metrics_name)
    with open(metrics_path, "w", encoding="utf-8") as f:
        json.dump(metrics, f, indent=2)

    return {
        "long_full_path": long_path,
        "parent_index_path": parent_path,
        "metrics_path": metrics_path,
        "metrics": metrics,
    }


In [12]:
result = build_parent_index_and_long_full_duckdb(
    df=data,          # or data_out after unify steps
    out_dir="outputs_new/parent",
    group_id_col="dup_group_id",  # or "dup_group_id_merged" if you renamed it
    date_col="date_first_seen",
)

result["metrics"]


{'total_rows': 8084860,
 'clustered_rows_has_dup_group_id': 1148031,
 'singleton_rows_no_dup_group_id': 6936829,
 'clusters_distinct_dup_group_id': 510687,
 'groups_full_clusters_plus_singletons': 7447516,
 'long_full_rows_written': 8084860,
 'parent_index_rows_written': 7447516}

In [17]:
result

{'long_full_path': 'outputs_new/parent/dedupe_clusters_long_full.parquet',
 'parent_index_path': 'outputs_new/parent/parent_version_index_full.parquet',
 'metrics_path': 'outputs_new/parent/dedupe_full_metrics.json',
 'metrics': {'total_rows': 8084860,
  'clustered_rows_has_dup_group_id': 1148031,
  'singleton_rows_no_dup_group_id': 6936829,
  'clusters_distinct_dup_group_id': 510687,
  'groups_full_clusters_plus_singletons': 7447516,
  'long_full_rows_written': 8084860,
  'parent_index_rows_written': 7447516}}

In [20]:
import duckdb

con = duckdb.connect()

long_path = "outputs_new/parent/dedupe_clusters_long_full.parquet"
parent_path = "outputs_new/parent/parent_version_index_full.parquet"


In [27]:
con.sql(f"""
SELECT *
FROM '{parent_path}'
ORDER BY RANDOM()
LIMIT 10
""").df()


,dup_group_id_full,parent_record_id,parent_doi,parent_server_name,parent_date_first_seen,parent_year_first_seen,version_record_ids,version_dois,servers_with_counts,total_versions,first_version_date,last_version_date,most_recent_version_servers
0,crossref::10.7554/elife.74550,crossref::10.7554/elife.74550,10.7554/elife.74550,eLife,2022-07-14,2022,crossref::10.7554/elife.74550,10.7554/elife.74550,eLife (1),1,2022-07-14,2022-07-14,eLife
1,datacite::10.48550/arxiv.2408.11865,datacite::10.48550/arxiv.2408.11865,10.48550/arxiv.2408.11865,arXiv,2024-08-23,2024,datacite::10.48550/arxiv.2408.11865,10.48550/arxiv.2408.11865,arXiv (1),1,2024-08-23,2024-08-23,arXiv
2,openalex::W1592208368,openalex::W1592208368,None,RePEc: Research Papers in Economics,2006-08-01,2006,crossref::10.2139/ssrn.1550662; openalex::W159...,10.2139/ssrn.1550662,RePEc: Research Papers in Economics (1); SSRN (1),2,2006-08-01,2007-01-01,SSRN
3,crossref::10.2139/ssrn.5179725,crossref::10.2139/ssrn.5179725,10.2139/ssrn.5179725,SSRN,2025-01-01,2025,crossref::10.2139/ssrn.5179725,10.2139/ssrn.5179725,SSRN (1),1,2025-01-01,2025-01-01,SSRN
4,openalex::W4381468740,openalex::W4381468740,None,HAL,2021-06-22,2021,openalex::W4381468740,None,HAL (1),1,2021-06-22,2021-06-22,HAL
5,datacite::10.48550/arxiv.2205.07112,datacite::10.48550/arxiv.2205.07112,10.48550/arxiv.2205.07112,arXiv,2022-05-17,2022,datacite::10.48550/arxiv.2205.07112,10.48550/arxiv.2205.07112,arXiv (1),1,2022-05-17,2022-05-17,arXiv
6,openalex::W2623528281,openalex::W2623528281,None,HAL,2016-04-01,2016,openalex::W2623528281,None,HAL (1),1,2016-04-01,2016-04-01,HAL
7,datacite::10.48550/arxiv.2211.09456,datacite::10.48550/arxiv.2211.09456,10.48550/arxiv.2211.09456,arXiv,2022-11-18,2022,datacite::10.48550/arxiv.2211.09456,10.48550/arxiv.2211.09456,arXiv (1),1,2022-11-18,2022-11-18,arXiv
8,openalex::W3121197868,openalex::W3121197868,None,HAL,2014-09-12,2014,openalex::W3121197868,None,HAL (1),1,2014-09-12,2014-09-12,HAL
9,datacite::10.48550/arxiv.0906.0292,datacite::10.48550/arxiv.0906.0292,10.48550/arxiv.0906.0292,arXiv,2022-03-15,2022,datacite::10.48550/arxiv.0906.0292,10.48550/arxiv.0906.0292,arXiv (1),1,2022-03-15,2022-03-15,arXiv


In [28]:
con.sql(f"""
SELECT *
FROM '{long_path}'
ORDER BY RANDOM()
LIMIT 10
""").df()


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup,dup_group_id_full,dt,parent_record_id_selected,parent_server_name_selected,parent_doi_selected,parent_date_selected,parent_year_selected,is_parent_selected,group_size
0,datacite::10.48550/arxiv.1106.5991,arXiv,datacite,10.48550/arxiv.1106.5991,https://doi.org/10.48550/arxiv.1106.5991,https://arxiv.org/abs/1106.5991,Energy transport through rare collisions,"Huveneers, François",None,None,"[{""relatedIdentifier"": ""10.1007/s10955-011-037...",2,,10.1007/s10955-011-0374-9,,,true,<NA>,<NA>,None,"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",parent,2022-03-13,2022,parent,None,None,francois_huveneers,energy transport through rare collisions,huveneers|f,None,parent,None,None,parent,None,None,datacite::10.48550/arxiv.1106.5991,2022-03-13,datacite::10.48550/arxiv.1106.5991,arXiv,10.48550/arxiv.1106.5991,2022-03-13,2022,True,1
1,datacite::10.48550/arxiv.2507.05061,arXiv,datacite,10.48550/arxiv.2507.05061,https://doi.org/10.48550/arxiv.2507.05061,https://arxiv.org/abs/2507.05061,Spin liquid state in a three-dimensional pyroc...,"Jena, U.; Kundu, S.; Lee, Suheon; Faure, Q.; D...",None,None,[],1,,,,,false,<NA>,<NA>,None,"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",parent,2025-07-08,2025,parent,None,None,None,None,None,None,parent,None,None,parent,None,None,datacite::10.48550/arxiv.2507.05061,2025-07-08,datacite::10.48550/arxiv.2507.05061,arXiv,10.48550/arxiv.2507.05061,2025-07-08,2025,True,1
2,datacite::10.22004/ag.econ.50718,AgEcon Search,datacite,10.22004/ag.econ.50718,https://doi.org/10.22004/ag.econ.50718,https://ageconsearch.umn.edu/record/50718,How Homophily Affects Learning and Diffusion i...,"Golub, Benjamin; Jackson, Matthew O.",None,None,[],None,,,,,false,<NA>,<NA>,None,"{""client"": {""data"": {""id"": ""tind.agecon"", ""typ...",parent,2019-08-24,2019,parent,None,None,benjamin_golub;jackson_matthew_o,how homophily affects learning and diffusion i...,golub|b;jackson|m,None,parent,None,None,parent,None,None,datacite::10.22004/ag.econ.50718,2019-08-24,datacite::10.22004/ag.econ.50718,AgEcon Search,10.22004/ag.econ.50718,2019-08-24,2019,True,1
3,openalex::W4311755543,HAL,openalex,None,None,https://univ-paris8.hal.science/hal-03062194,Museography of the Contemporary,Lionel Ruffel,None,None,None,None,None,None,None,None,None,<NA>,<NA>,None,None,parent,2015-05-06,2015,parent,None,None,lionel_ruffel,museography of the contemporary,ruffel|l,None,parent,None,None,parent,None,None,openalex::W4311755543,2015-05-06,openalex::W4311755543,HAL,None,2015-05-06,2015,True,1
4,datacite::10.48550/arxiv.1505.01008,arXiv,datacite,10.48550/arxiv.1505.01008,https://doi.org/10.48550/arxiv.1505.01008,https://arxiv.org/abs/1505.01008,Physics cross sections and event generation of...,"Mo, Xin; Li, Gang; Ruan, Manqi; Lou, Xinchou",None,None,"[{""relatedIdentifier"": ""10.1088/1674-1137/40/3...",2,,10.1088/1674-1137/40/3/033001,,,true,<NA>,<NA>,None,"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",parent,2022-03-08,2022,parent,None,None,None,None,None,None,parent,None,None,parent,None,None,datacite::10.48550/arxiv.1505.01008,2022-03-08,datacite::10.48550/arxiv.1505.01008,arXiv,10.48550/arxiv.1505.01008,2022-03-08,2022,True,1
5,openalex::W3122587013,RePEc: Research Papers in Economics,openalex,None,None,https://econpapers.repec.org/RePEc:

In [21]:
con.sql(f"""
SELECT dup_group_id_full, COUNT(*) AS n
FROM '{long_path}'
GROUP BY 1
ORDER BY n DESC
LIMIT 20
""").df()


,dup_group_id_full,n
0,datacite::10.5281/zenodo.14289192,702
1,datacite::10.5281/zenodo.15609432,569
2,datacite::10.22004/ag.econ.138684,450
3,datacite::10.22004/ag.econ.136112,390
4,datacite::10.22004/ag.econ.286599,301
5,datacite::10.5281/zenodo.15832876,286
6,datacite::10.5281/zenodo.15161515,261
7,crossref::10.31219/osf.io/ed2a9,210
8,crossref::10.32388/174914,184
9,datacite::10.5281/zenodo.6784019,182


In [30]:
con.sql(f"""
SELECT *
FROM '{long_path}'
WHERE dup_group_id_full IN (
    SELECT DISTINCT dup_group_id_full
    FROM '{long_path}'
    ORDER BY RANDOM()
    LIMIT 5
)
""").df()


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup,dup_group_id_full,dt,parent_record_id_selected,parent_server_name_selected,parent_doi_selected,parent_date_selected,parent_year_selected,is_parent_selected,group_size
0,datacite::10.48550/arxiv.1908.00749,arXiv,datacite,10.48550/arxiv.1908.00749,https://doi.org/10.48550/arxiv.1908.00749,https://arxiv.org/abs/1908.00749,The best extending cover-preserving geometric ...,"He, Peng; Wang, Xue-ping",None,None,"[{""relatedIdentifier"": ""10.1007/s10114-023-153...",2,,10.1007/s10114-023-1531-1,,,true,<NA>,<NA>,None,"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",parent,2022-02-27,2022,parent,None,None,he_peng;ping_wang_xue,the best extending cover preserving geometric ...,he|p;wang|x,None,parent,None,None,parent,None,None,datacite::10.48550/arxiv.1908.00749,2022-02-27,datacite::10.48550/arxiv.1908.00749,arXiv,10.48550/arxiv.1908.00749,2022-02-27,2022,True,1
1,crossref::10.7554/elife.26851,eLife,crossref,10.7554/elife.26851,https://doi.org/10.7554/elife.26851,https://elifesciences.org/articles/26851,Molecular architecture underlying fluid absorp...,"Honda, Keiji; Kim, Sung Huhn; Kelly, Michael C...","Molecular Biology and Genetics Section, Nation...",None,"{""is-supplemented-by"": [{""asserted-by"": ""subje...",None,,,,,false,<NA>,<NA>,None,None,parent,2017-09-28,2017,parent,None,crossref::10.7554/elife.26851,andrew_griffith_j;burns_c_joseph;c_kelly_micha...,molecular architecture underlying fluid absorp...,None,None,parent - duplicate (eLife),crossref::10.7554/elife.26851.034,fuzzy::tokenbag::andrew_griffith_j;burns_c_jos...,parent - duplicate (eLife),crossref::10.7554/elife.26851.034,fuzzy::tokenbag::andrew_griffith_j;burns_c_jos...,crossref::10.7554/elife.26851,2017-09-28,crossref::10.7554/elife.26851,eLife,10.7554/elife.26851,2017-09-28,2017,True,1
2,datacite::10.48550/arxiv.2203.05536,arXiv,datacite,10.48550/arxiv.2203.05536,https://doi.org/10.48550/arxiv.2203.05536,https://arxiv.org/abs/2203.05536,"Non-zero $θ_{13}$, CP-violation and Neutrinole...","Barman, Animesh; Francis, Ng. K.; Thapa, Bikas...",None,None,"[{""relatedIdentifier"": ""10.1142/s0217751x23500...",3,,10.1142/s0217751x23500124,,,true,<NA>,<NA>,None,"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",parent,2022-03-11,2022,parent,None,None,None,None,None,None,parent,None,None,parent,None,None,datacite::10.48550/arxiv.2203.05536,2022-03-11,datacite::10.48550/arxiv.2203.05536,arXiv,10.48550/arxiv.2203.05536,2022-03-11,2022,True,1
3,openalex::W4392474942,Munich Personal RePEc Archive,openalex,None,None,http://www.theses.fr/2015ENMP0075/document,Mixed titanium and tin oxide coprecipitation i...,Alexandre Pichavant,None,None,None,None,None,None,None,None,None,<NA>,<NA>,None,None,parent,2015-12-17,2015,parent,None,None,None,None,None,None,parent,None,None,parent,None,None,openalex::W4392474942,2015-12-17,openalex::W4392474942,Munich Personal RePEc Archive,None,2015-12-17,2015,True,1
4,datacite::10.48550/arxiv.2011.07907,arXiv,datacite,10.48550/arxiv.2011.07907,https://doi.org/10.48550/arxiv.2011.07907,https://arxiv.org/abs/2011.07907,Strong diffusion approximation in averaging an...,"Kifer, Yuri",None,None,[],7,,,,,false,<NA>,<NA>,None,"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",parent,2022-02-24,2022,parent,None,None,kifer_yuri,strong diffusion approx